In [6]:
# conda deactivate && conda env update --file glider-utils/environment.yml --prune
# from subprocess import run
# run(["/home/sam_woodman_noaa_gov/glider-utils/resources/sync-cache.sh"])

In [20]:
import os
from pyglider import slocum
from pyglider import utils
import numpy as np
# import pandas as pd
import xarray as xr
from esdglider.profiles import get_profiles_esd

In [14]:
cacdir = "../resources/example-data/cache"
binarydir = "../resources/example-data/binary-delayed"
deploymentyaml = "../resources/example-data/amlr08-20220513-delayed.yml"
deploymentyaml_eng = "../resources/deployment-eng.yml"
outdir = "../resources/example-data/out"

In [5]:
l1ts_outname_sci = slocum.binary_to_timeseries(
    binarydir, cacdir, outdir, deploymentyaml,
    search='*.[D|E|d|e]bd', fnamesuffix='-sci',
    time_base='sci_water_temp', profile_filt_time=100,
    profile_min_time=300, maxgap=300)

/opt/conda/envs/esdgliderutils/lib/python3.12/site-packages/pyglider/utils.py:529: RuntimeWarning: invalid value encountered in divide
  deg = (np.fix(nmea / 100) +
/opt/conda/envs/esdgliderutils/lib/python3.12/site-packages/pyglider/utils.py:530: RuntimeWarning: invalid value encountered in remainder
  np.sign(nmea) * np.remainder(np.abs(nmea), 100) / 60)
/opt/conda/envs/esdgliderutils/lib/python3.12/site-packages/pyglider/utils.py:530: RuntimeWarning: invalid value encountered in multiply
  np.sign(nmea) * np.remainder(np.abs(nmea), 100) / 60)
/opt/conda/envs/esdgliderutils/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:1515: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
/opt/conda/envs/esdgliderutils/lib/python3.12/site-packages/pyglider/utils.py:584: RuntimeWarning: invalid value encountered in multiply
  return val * 10
/opt/conda/envs/esdgliderutils/lib/python3.12/site-packages/gsw/_wrapped_ufuncs.py:7991: RuntimeWarning: inva

In [17]:
l1ts_outname_eng = slocum.binary_to_timeseries(
    binarydir, cacdir, outdir, 
    [deploymentyaml, deploymentyaml_eng],
    search='*.[D|E|d|e]bd', fnamesuffix='-eng',
    # search='*.[D|E]BD', fnamesuffix='',
    time_base='m_depth', profile_filt_time=100,
    profile_min_time=300, maxgap=300)

/opt/conda/envs/esdgliderutils/lib/python3.12/site-packages/numpy/lib/_function_base_impl.py:1515: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])


In [ ]:
l1ts_outname_sci

In [8]:
ts_sci = xr.open_dataset(l1ts_outname_sci)
# ts_oxy = xr.open_dataset(os.path.join(l1tsdir, f"{deployment}-oxy.nc"))
ts_sci
# list(ts_sci.data_vars)

<xarray.Dataset> Size: 22MB
Dimensions:                (time: 113607)
Coordinates:
  * time                   (time) datetime64[ns] 909kB 2022-05-13T18:13:38.68...
Data variables: (12/23)
    latitude               (time) float64 909kB ...
    longitude              (time) float64 909kB ...
    m_depth                (time) float64 909kB ...
    heading                (time) float64 909kB ...
    pitch                  (time) float64 909kB ...
    roll                   (time) float64 909kB ...
    ...                     ...
    salinity               (time) float64 909kB ...
    potential_density      (time) float64 909kB ...
    density                (time) float64 909kB ...
    potential_temperature  (time) float64 909kB ...
    profile_index          (time) float64 909kB ...
    profile_direction      (time) float64 909kB ...
Attributes: (12/63)
    Conventions:               CF-1.8
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0
    acknowledgment:            This work was supported by funding from NOAA.
    acoustics:                 {'make_model': 'NORTEK Inc Signature 100 Compa...
    cdm_data_type:             Trajectory
    comment:                   TrajectoryProfile
    ...                        ...
    time_coverage_start:       2022-05-13T18:13:38.687000064
    title:                     amlr08amlr08-20220513T1813
    transmission_system:       IRIDIUM
    wmo_id:                     
    deployment_start:          2022-05-13T18:13:3
    deployment_end:            2022-05-13T18:13:3

In [13]:
ts_eng = xr.open_dataset(l1ts_outname_eng)
ts_eng

<xarray.Dataset> Size: 10MB
Dimensions:                (time: 60002)
Coordinates:
  * time                   (time) datetime64[ns] 480kB 2022-05-13T18:13:42.53...
Data variables: (12/19)
    latitude               (time) float64 480kB ...
    longitude              (time) float64 480kB ...
    m_depth                (time) float64 480kB ...
    heading                (time) float64 480kB ...
    pitch                  (time) float64 480kB ...
    roll                   (time) float64 480kB ...
    ...                     ...
    salinity               (time) float64 480kB ...
    potential_density      (time) float64 480kB ...
    density                (time) float64 480kB ...
    potential_temperature  (time) float64 480kB ...
    profile_index          (time) float64 480kB ...
    profile_direction      (time) float64 480kB ...
Attributes: (12/63)
    Conventions:               CF-1.8
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0
    acknowledgment:            This work was supported by funding from NOAA.
    acoustics:                 {'make_model': 'NORTEK Inc Signature 100 Compa...
    cdm_data_type:             Trajectory
    comment:                   TrajectoryProfile
    ...                        ...
    time_coverage_start:       2022-05-13T18:13:42.536999936
    title:                     amlr08amlr08-20220513T1813
    transmission_system:       IRIDIUM
    wmo_id:                     
    deployment_start:          2022-05-13T18:13:4
    deployment_end:            2022-05-13T18:13:4

In [18]:
ts_sci.close()
# ts_oxy.close()
ts_eng.close()

## Testing times

Main purpose of this section is to confirm that xarray.merge is doing what we expect

In [24]:
t1 = ts_sci.time.to_numpy()
# t2 = ts_oxy.time.to_numpy()
t3 = ts_eng.time.to_numpy()

df_union = np.union1d(t1, t3)
print(len(t1))
print(len(t3))
print(len(df_union))
df_union

113607
60002
173277


array(['1970-01-01T00:00:00.000000000', '1970-01-01T00:00:00.004262500',
       '2001-09-09T01:46:40.000000000', ...,
       '2022-05-16T18:29:22.747000064', '2022-05-16T18:29:26.811000064',
                                 'NaT'], dtype='datetime64[ns]')

## Combining datasets

In [9]:
vars_all = ['latitude', 'longitude', 'waypoint_latitude', 'waypoint_longitude']
sci_vars = vars_all + ['conductivity', 'temperature', 'pressure',
            'depth', 'salinity', 'potential_density', 'density', 'potential_temperature',
            'profile_index', 'profile_direction']
oxy_vars = vars_all + ["oxygen_concentration"]
eng_vars = vars_all + ["m_depth", "heading", "pitch", "roll"]

In [10]:
ts_sci = ts_sci[sci_vars]
ts_oxy = ts_oxy[oxy_vars]
ts_eng = ts_eng[eng_vars]

In [ ]:
ts = xr.merge([ts_sci, ts_oxy, ts_eng], compat = "no_conflicts", 
              join  = "outer", combine_attrs = "override")
ts

In [ ]:
# any(np.isnan(ts.latitude.values))
# good = ~np.isnan(ts.latitude.values + ts.longitude.values)

# print(np.nanmax(ts.latitude.values))
# print(np.max(ts.latitude.values[good]))

Recalculate attributes as necessary

In [ ]:
good = ~np.isnan(ts.latitude.values + ts.longitude.values)
ts.attrs['geospatial_lat_max'] = np.nanmax(ts.latitude.values[good])
ts.attrs['geospatial_lat_min'] = np.nanmin(ts.latitude.values[good])
ts.attrs['geospatial_lon_max'] = np.nanmax(ts.longitude.values[good])
ts.attrs['geospatial_lon_min'] = np.nanmin(ts.longitude.values[good])
ts.attrs['geospatial_lat_units'] = 'degrees_north'
ts.attrs['geospatial_lon_units'] = 'degrees_east'

dt = ts.time.values
ts.attrs['time_coverage_start'] = '%s' % dt[0]
ts.attrs['time_coverage_end'] = '%s' % dt[-1]

ts.attrs['deployment_start'] = str(dt[0].astype('datetime64[s]'))
ts.attrs['deployment_end'] = str(dt[-1].astype('datetime64[s]'))

Recalculate profile values as necessary

In [ ]:
depth_good = np.where(~np.isnan(ts.m_depth))[0]
ts.m_depth.values = np.interp(np.arange(len(ts.m_depth)), good, ts.m_depth.values[good])
ts = get_profiles_esd(ts, "m_depth")

# tg_ind = utils.find_gaps(ts.time.values[depth_good], ts.time.values, 300)
# np.where(tg_ind)
# dep = ts.m_depth.to_pandas()

ts = utils.get_distance_over_ground(ts)
ts

In [ ]:
outname = os.path.join(l1tsdir, f"{deployment}-union.nc")
ts.to_netcdf(outname, 'w',
             encoding={'time': {'units': 'seconds since 1970-01-01T00:00:00Z',
                                '_FillValue': np.nan,
                                'dtype': 'float64'}})